In [2]:
import pandas as pd
import numpy as np


In [3]:
customers = pd.read_csv(r"C:\Users\KIIT\Desktop\Professional\Projects\BFSI Project\Datasets\customers.csv")
credit = pd.read_csv(r"C:\Users\KIIT\Desktop\Professional\Projects\BFSI Project\Datasets\credit_profiles.csv")
transactions = pd.read_csv(r"C:\Users\KIIT\Desktop\Professional\Projects\BFSI Project\Datasets\transactions.csv")
repayments = pd.read_csv(r"C:\Users\KIIT\Desktop\Professional\Projects\BFSI Project\Datasets\repayments.csv")


## Initial Data Quality Checks

In [4]:
customers.info()
credit.info()
transactions.info()
repayments.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            2000 non-null   int64 
 1   age                    2000 non-null   int64 
 2   gender                 2000 non-null   object
 3   city                   2000 non-null   object
 4   account_tenure_months  2000 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 78.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit_profile_id  2000 non-null   int64  
 1   customer_id        2000 non-null   int64  
 2   credit_limit       2000 non-null   int64  
 3   credit_score       1839 non-null   float64
 4   utilization_ratio  2000 non-null   float64
dtypes: float64(2), int64(3)
memory usage: 7

#### 1. Basic Shape & Structure Check

In [6]:
def basic_shape_check(df, name):
    print(f"\n{name} — Shape")
    print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

In [7]:
basic_shape_check(customers, "Customers")
basic_shape_check(credit, "Credit Profiles")
basic_shape_check(transactions, "Transactions")
basic_shape_check(repayments, "Repayments")


Customers — Shape
Rows: 2000, Columns: 5

Credit Profiles — Shape
Rows: 2000, Columns: 5

Transactions — Shape
Rows: 12000, Columns: 5

Repayments — Shape
Rows: 5000, Columns: 5


#### 2. Schema & Datatype Validation

In [8]:
def schema_check(df, name):
    print(f"\n{name} — Schema Info")
    print(df.info())


In [9]:
schema_check(customers, "Customers")
schema_check(credit, "Credit Profiles")
schema_check(transactions, "Transactions")
schema_check(repayments, "Repayments")



Customers — Schema Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            2000 non-null   int64 
 1   age                    2000 non-null   int64 
 2   gender                 2000 non-null   object
 3   city                   2000 non-null   object
 4   account_tenure_months  2000 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 78.3+ KB
None

Credit Profiles — Schema Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit_profile_id  2000 non-null   int64  
 1   customer_id        2000 non-null   int64  
 2   credit_limit       2000 non-null   int64  
 3   credit_score       1839 non-null   float64
 4   utilization_ratio  2000 no

#### 3. Missing Value Assessment

In [10]:
def missing_value_check(df, name):
    print(f"\n{name} — Missing Values")
    missing = df.isna().sum()
    missing_pct = (missing / len(df)) * 100
    result = pd.DataFrame({
        "missing_count": missing,
        "missing_pct": missing_pct.round(2)
    })
    print(result[result["missing_count"] > 0])


In [11]:
missing_value_check(customers, "Customers")
missing_value_check(credit, "Credit Profiles")
missing_value_check(transactions, "Transactions")
missing_value_check(repayments, "Repayments")



Customers — Missing Values
Empty DataFrame
Columns: [missing_count, missing_pct]
Index: []

Credit Profiles — Missing Values
              missing_count  missing_pct
credit_score            161         8.05

Transactions — Missing Values
Empty DataFrame
Columns: [missing_count, missing_pct]
Index: []

Repayments — Missing Values
Empty DataFrame
Columns: [missing_count, missing_pct]
Index: []


#### 4. Duplicate Record Check

In [12]:
def duplicate_check(df, name):
    dup_count = df.duplicated().sum()
    print(f"\n{name} — Duplicate Rows: {dup_count}")


In [13]:
duplicate_check(customers, "Customers")
duplicate_check(credit, "Credit Profiles")
duplicate_check(transactions, "Transactions")
duplicate_check(repayments, "Repayments")



Customers — Duplicate Rows: 0

Credit Profiles — Duplicate Rows: 0

Transactions — Duplicate Rows: 0

Repayments — Duplicate Rows: 0


#### 5. Primary Key Uniqueness Check

In [14]:
def primary_key_check(df, pk, name):
    total = df.shape[0]
    unique = df[pk].nunique()
    print(f"\n{name} — PK Check ({pk})")
    print(f"Total Rows: {total}")
    print(f"Unique {pk}: {unique}")


In [15]:
primary_key_check(customers, "customer_id", "Customers")
primary_key_check(credit, "credit_profile_id", "Credit Profiles")
primary_key_check(transactions, "transaction_id", "Transactions")
primary_key_check(repayments, "repayment_id", "Repayments")



Customers — PK Check (customer_id)
Total Rows: 2000
Unique customer_id: 2000

Credit Profiles — PK Check (credit_profile_id)
Total Rows: 2000
Unique credit_profile_id: 2000

Transactions — PK Check (transaction_id)
Total Rows: 12000
Unique transaction_id: 12000

Repayments — PK Check (repayment_id)
Total Rows: 5000
Unique repayment_id: 5000


#### 6. Foreign Key Integrity Check (Python Side)

In [16]:
def foreign_key_check(child_df, parent_df, fk, pk, name):
    missing_fk = child_df[~child_df[fk].isin(parent_df[pk])]
    print(f"\n{name} — FK Integrity")
    print(f"Orphan Records: {missing_fk.shape[0]}")


In [17]:
foreign_key_check(credit, customers, "customer_id", "customer_id", "Credit → Customers")
foreign_key_check(transactions, customers, "customer_id", "customer_id", "Transactions → Customers")
foreign_key_check(repayments, customers, "customer_id", "customer_id", "Repayments → Customers")



Credit → Customers — FK Integrity
Orphan Records: 0

Transactions → Customers — FK Integrity
Orphan Records: 0

Repayments → Customers — FK Integrity
Orphan Records: 0


#### 7. Quick Statistical Sanity Check (Numerical Columns)

In [18]:
def numeric_summary(df, cols, name):
    print(f"\n{name} — Numeric Summary")
    print(df[cols].describe())


In [19]:
numeric_summary(credit, ["credit_limit", "credit_score", "utilization_ratio"], "Credit Profiles")
numeric_summary(transactions, ["transaction_amount"], "Transactions")
numeric_summary(repayments, ["payment_amount", "payment_delay_days"], "Repayments")



Credit Profiles — Numeric Summary
        credit_limit  credit_score  utilization_ratio
count    2000.000000   1839.000000        2000.000000
mean   144712.500000    606.767809           0.624025
std     84817.904255    176.496477           0.329189
min     50000.000000    300.000000           0.050000
25%     75000.000000    455.000000           0.350000
50%    100000.000000    606.000000           0.620000
75%    200000.000000    764.000000           0.900000
max    300000.000000    899.000000           1.200000

Transactions — Numeric Summary
       transaction_amount
count        12000.000000
mean          3677.339170
std           5413.978212
min              0.330000
25%           1029.735000
50%           2438.685000
75%           4918.777500
max         237645.000000

Repayments — Numeric Summary
       payment_amount  payment_delay_days
count     5000.000000         5000.000000
mean     10191.768306            8.695000
std       5592.197310           10.323434
min        513.

#### 8. Categorical Sanity Check

In [20]:
def category_check(df, col, name):
    print(f"\n{name} — {col} Distribution")
    print(df[col].value_counts().head(10))


In [21]:
category_check(transactions, "transaction_category", "Transactions")
category_check(customers, "city", "Customers")



Transactions — transaction_category Distribution
transaction_category
grocery            1282
Utilities          1253
Dining             1239
Online Shopping    1232
ENTERTAINMENT      1218
entertainment      1187
Fuel               1174
Electronics        1161
Travel             1128
Groceries          1126
Name: count, dtype: int64

Customers — city Distribution
city
Delhi        312
Hyderabad    302
Bengaluru    294
Chennai      275
Mumbai       273
Kolkata      272
Pune         272
Name: count, dtype: int64


## Handling Missing Values

#### For customers table

In [22]:
# Check
customers.isna().sum()


customer_id              0
age                      0
gender                   0
city                     0
account_tenure_months    0
dtype: int64

In [23]:
# Conservative handling
customers["age"] = customers["age"].fillna(customers["age"].median())
customers["gender"] = customers["gender"].fillna("unknown")
customers["city"] = customers["city"].fillna("unknown")
customers["account_tenure_months"] = customers["account_tenure_months"].fillna(
    customers["account_tenure_months"].median()
)


#### For credit_profiles

In [24]:
credit["credit_score"] = credit["credit_score"].fillna(
    credit["credit_score"].median()
)


In [25]:
credit["credit_limit"] = credit["credit_limit"].fillna(
    credit["credit_limit"].median()
)

credit["utilization_ratio"] = credit["utilization_ratio"].fillna(0)


#### For transactions

In [26]:
transactions["transaction_category"] = (
    transactions["transaction_category"]
    .fillna("unknown")
)


In [27]:
transactions["transaction_amount"] = transactions["transaction_amount"].fillna(0)


#### For repayments

In [28]:
repayments["payment_delay_days"] = repayments["payment_delay_days"].fillna(0)


In [29]:
repayments["payment_amount"] = repayments["payment_amount"].fillna(0)


#### Validation after Imputation

In [30]:
print("Customers:\n", customers.isna().sum())
print("\nCredit Profiles:\n", credit.isna().sum())
print("\nTransactions:\n", transactions.isna().sum())
print("\nRepayments:\n", repayments.isna().sum())


Customers:
 customer_id              0
age                      0
gender                   0
city                     0
account_tenure_months    0
dtype: int64

Credit Profiles:
 credit_profile_id    0
customer_id          0
credit_limit         0
credit_score         0
utilization_ratio    0
dtype: int64

Transactions:
 transaction_id          0
customer_id             0
transaction_date        0
transaction_category    0
transaction_amount      0
dtype: int64

Repayments:
 repayment_id          0
customer_id           0
payment_date          0
payment_amount        0
payment_delay_days    0
dtype: int64


## Datatype Standardization

#### Standardize Identifier Columns (Keys)
- IDs should never be float and should remain stable for joins.

In [31]:
customers["customer_id"] = customers["customer_id"].astype("int64")
credit["credit_profile_id"] = credit["credit_profile_id"].astype("int64")
credit["customer_id"] = credit["customer_id"].astype("int64")

transactions["transaction_id"] = transactions["transaction_id"].astype("int64")
transactions["customer_id"] = transactions["customer_id"].astype("int64")

repayments["repayment_id"] = repayments["repayment_id"].astype("int64")
repayments["customer_id"] = repayments["customer_id"].astype("int64")


#### Standardize Date & Time Columns
Datetime types are required for:

- Trend analysis

- Monthly aggregation

- Churn signals

In [32]:
transactions["transaction_date"] = pd.to_datetime(
    transactions["transaction_date"],
    errors="coerce"
)

repayments["payment_date"] = pd.to_datetime(
    repayments["payment_date"],
    errors="coerce"
)

# errors="coerce" ensures any unexpected format becomes NaT instead of breaking the pipeline.

#### Standardize Numeric Financial Columns
- All monetary values must support arithmetic without surprises.

In [33]:
credit["credit_limit"] = credit["credit_limit"].astype("float64")
credit["credit_score"] = credit["credit_score"].astype("float64")
credit["utilization_ratio"] = credit["utilization_ratio"].astype("float64")

transactions["transaction_amount"] = transactions["transaction_amount"].astype("float64")

repayments["payment_amount"] = repayments["payment_amount"].astype("float64")
repayments["payment_delay_days"] = repayments["payment_delay_days"].astype("int64")


#### Standardize Categorical Columns
- Consistent strings improve grouping, joins, and BI visuals.

In [34]:
customers["gender"] = customers["gender"].astype("string")
customers["city"] = customers["city"].astype("string")

transactions["transaction_category"] = transactions["transaction_category"].astype("string")


#### Final Schema Validation

In [35]:
print("Customers schema")
print(customers.dtypes)

print("\nCredit Profiles schema")
print(credit.dtypes)

print("\nTransactions schema")
print(transactions.dtypes)

print("\nRepayments schema")
print(repayments.dtypes)


Customers schema
customer_id                       int64
age                               int64
gender                   string[python]
city                     string[python]
account_tenure_months             int64
dtype: object

Credit Profiles schema
credit_profile_id      int64
customer_id            int64
credit_limit         float64
credit_score         float64
utilization_ratio    float64
dtype: object

Transactions schema
transaction_id                   int64
customer_id                      int64
transaction_date        datetime64[ns]
transaction_category    string[python]
transaction_amount             float64
dtype: object

Repayments schema
repayment_id                   int64
customer_id                    int64
payment_date          datetime64[ns]
payment_amount               float64
payment_delay_days             int64
dtype: object


## Transaction Category Normalization

#### Initial Sanity Check (Before Normalization)
To understand how messy the categories actually are.

In [36]:
transactions["transaction_category"].value_counts()


transaction_category
grocery            1282
Utilities          1253
Dining             1239
Online Shopping    1232
ENTERTAINMENT      1218
entertainment      1187
Fuel               1174
Electronics        1161
Travel             1128
Groceries          1126
Name: count, dtype: Int64

#### 1. Basic Text Standardization

In [37]:
transactions["transaction_category"] = (
    transactions["transaction_category"]
    .str.strip()        # remove leading/trailing spaces
    .str.lower()        # normalize casing
)


#### 2. Define Business-Approved Category Mapping

In [38]:
category_mapping = {
    "grocery": "groceries",
    "groceries": "groceries",

    "entertainment": "entertainment",
    "entertainment ": "entertainment",

    "online shopping": "online shopping",
    "online_shopping": "online shopping",

    "fuel": "fuel",
    "petrol": "fuel",

    "dining": "dining",
    "restaurant": "dining",

    "travel": "travel",

    "electronics": "electronics",

    "utilities": "utilities",

    "unknown": "unknown"
}


#### 3. Apply the Mapping

In [39]:
transactions["transaction_category"] = (
    transactions["transaction_category"]
    .map(category_mapping)
    .fillna("others")
)


#### 4. Post-Normalization Validation

In [40]:
transactions["transaction_category"].value_counts()


transaction_category
groceries          2408
entertainment      2405
utilities          1253
dining             1239
online shopping    1232
fuel               1174
electronics        1161
travel             1128
Name: count, dtype: int64

## Duplicate Record Handling

#### Identify Exact Duplicate Rows (Quick Check)

In [41]:
print("Exact duplicate rows:")
print("Customers:", customers.duplicated().sum())
print("Credit Profiles:", credit.duplicated().sum())
print("Transactions:", transactions.duplicated().sum())
print("Repayments:", repayments.duplicated().sum())


Exact duplicate rows:
Customers: 0
Credit Profiles: 0
Transactions: 0
Repayments: 0


#### Business-Level Duplicate Handling (Core Logic)

In [42]:
customers = customers.drop_duplicates(subset=["customer_id"])


In [43]:
credit = credit.drop_duplicates(subset=["customer_id"])


In [44]:
transactions = transactions.drop_duplicates(
    subset=[
        "customer_id",
        "transaction_date",
        "transaction_category",
        "transaction_amount"
    ]
)


In [45]:
repayments = repayments.drop_duplicates(
    subset=[
        "customer_id",
        "payment_date",
        "payment_amount"
    ]
)


#### Post-Deduplication Validation

In [46]:
print("After deduplication:")
print("Customers:", customers.shape[0])
print("Credit Profiles:", credit.shape[0])
print("Transactions:", transactions.shape[0])
print("Repayments:", repayments.shape[0])


After deduplication:
Customers: 2000
Credit Profiles: 2000
Transactions: 12000
Repayments: 5000


## Outlier Detection 

#### Transaction Amount Distribution (Global View)
Understand overall skew and magnitude of extreme spends.

In [47]:
transactions["transaction_amount"].describe(percentiles=[0.90, 0.95, 0.99])


count     12000.000000
mean       3677.339170
std        5413.978212
min           0.330000
50%        2438.685000
90%        8300.584000
95%       10774.273000
99%       17056.751500
max      237645.000000
Name: transaction_amount, dtype: float64

#### High-Value Transaction Thresholds (Quantile-Based)
Define business-safe outlier thresholds.

In [48]:
p95_amt = transactions["transaction_amount"].quantile(0.95)
p99_amt = transactions["transaction_amount"].quantile(0.99)

print("95th percentile:", p95_amt)
print("99th percentile:", p99_amt)


95th percentile: 10774.273
99th percentile: 17056.751500000002


#### Flag High-Value Transactions (No Deletion)
- Preserves valuable customers

- Enables targeted retention logic

- Avoids naive outlier removal

In [49]:
transactions["high_value_txn_flag"] = (
    transactions["transaction_amount"] > p99_amt
)


#### Customer-Level Spend Outliers
Detect customers with abnormally high total spend.

In [50]:
customer_spend = (
    transactions
    .groupby("customer_id")["transaction_amount"]
    .sum()
    .reset_index(name="total_spend")
)


In [51]:
customer_spend["total_spend"].describe(percentiles=[0.90, 0.95, 0.99])


count      1998.000000
mean      22086.121141
std       15706.935183
min          44.010000
50%       19809.585000
90%       38285.007000
95%       46042.272000
99%       61248.329000
max      249957.820000
Name: total_spend, dtype: float64

In [52]:
# Flag High-Valued Customers

p99_cust_spend = customer_spend["total_spend"].quantile(0.99)

customer_spend["high_value_customer_flag"] = (
    customer_spend["total_spend"] > p99_cust_spend
)


#### Transaction Frequency Outliers
Identify customers with unusually high transaction counts.

In [53]:
txn_freq = (
    transactions
    .groupby("customer_id")
    .size()
    .reset_index(name="transaction_count")
)


In [54]:
txn_freq["transaction_count"].describe(percentiles=[0.90, 0.95, 0.99])


count    1998.000000
mean        6.006006
std         2.445390
min         1.000000
50%         6.000000
90%         9.000000
95%        10.000000
99%        12.000000
max        16.000000
Name: transaction_count, dtype: float64

In [55]:
p99_txn_count = txn_freq["transaction_count"].quantile(0.99)

txn_freq["high_frequency_flag"] = (
    txn_freq["transaction_count"] > p99_txn_count
)


#### Credit Utilization Outliers
High utilization is a risk & churn signal, not an error.

In [56]:
credit["utilization_ratio"].describe()


count    2000.000000
mean        0.624025
std         0.329189
min         0.050000
25%         0.350000
50%         0.620000
75%         0.900000
max         1.200000
Name: utilization_ratio, dtype: float64

In [57]:
credit["high_utilization_flag"] = (
    credit["utilization_ratio"] > 0.8
)


#### Repayment Delay Outliers
Detect customers with severe or repeated delays.

In [58]:
repayments["payment_delay_days"].describe(percentiles=[0.90, 0.95, 0.99])


count    5000.000000
mean        8.695000
std        10.323434
min         0.000000
50%         5.000000
90%        30.000000
95%        30.000000
99%        30.000000
max        30.000000
Name: payment_delay_days, dtype: float64

In [59]:
repayments["severe_delay_flag"] = (
    repayments["payment_delay_days"] >= 30
)


#### Consolidate Outlier Signals (Preview)

In [60]:
outlier_summary = (
    customer_spend
    .merge(txn_freq, on="customer_id", how="left")
    .merge(
        credit[["customer_id", "high_utilization_flag"]],
        on="customer_id",
        how="left"
    )
)


In [61]:
outlier_summary.head()


,customer_id,total_spend,high_value_customer_flag,transaction_count,high_frequency_flag,high_utilization_flag
0,100000,31103.07,False,8,False,False
1,100001,9410.15,False,3,False,True
2,100002,10138.70,False,3,False,False
3,100003,14889.49,False,7,False,True
4,100004,8275.33,False,5,False,False


#### Key BFSI Principle (Important)

We do NOT remove:

- High spenders

- High frequency users

- High utilization customers

Because:

- They often generate maximum revenue

- They may have higher churn risk

- Removing them destroys business value

## Feature Engineering

#### Transaction-Level Feature Engineering → Customer Level
Convert raw transactions into behavioral features per customer.

In [62]:
txn_features = (
    transactions
    .groupby("customer_id")
    .agg(
        total_spend=("transaction_amount", "sum"),
        avg_transaction_value=("transaction_amount", "mean"),
        max_transaction_value=("transaction_amount", "max"),
        transaction_count=("transaction_amount", "count"),
        high_value_txn_count=("high_value_txn_flag", "sum")
    )
    .reset_index()
)


Why these features matter

- total_spend → revenue contribution

- avg_transaction_value → spending style

- transaction_count → engagement

- high_value_txn_count → premium behavior

#### Monthly Trend Features (Churn Signal)
Detect spend decline, a classic churn indicator.

In [63]:
transactions["year_month"] = transactions["transaction_date"].dt.to_period("M")


In [64]:
monthly_spend = (
    transactions
    .groupby(["customer_id", "year_month"])["transaction_amount"]
    .sum()
    .reset_index()
)


In [65]:
monthly_trend = (
    monthly_spend
    .sort_values(["customer_id", "year_month"])
    .groupby("customer_id")
    .agg(
        avg_monthly_spend=("transaction_amount", "mean"),
        last_month_spend=("transaction_amount", "last"),
        first_month_spend=("transaction_amount", "first")
    )
    .reset_index()
)


In [66]:
monthly_trend["spend_change_ratio"] = (
    (monthly_trend["last_month_spend"] - monthly_trend["first_month_spend"]) /
    monthly_trend["first_month_spend"].replace(0, 1)
)


#### Category Behavior Features
Understand spend mix & dependency.

In [67]:
category_features = (
    transactions
    .groupby(["customer_id", "transaction_category"])["transaction_amount"]
    .sum()
    .unstack(fill_value=0)
)


In [68]:
category_features.columns = [
    f"spend_{col}" for col in category_features.columns
]


In [69]:
category_features.reset_index(inplace=True)


#### Repayment Behavior Features (Churn & Risk)

In [70]:
repayment_features = (
    repayments
    .groupby("customer_id")
    .agg(
        total_repaid=("payment_amount", "sum"),
        avg_payment_delay=("payment_delay_days", "mean"),
        max_payment_delay=("payment_delay_days", "max"),
        delayed_payment_count=("payment_delay_days", lambda x: (x > 0).sum()),
        severe_delay_count=("severe_delay_flag", "sum")
    )
    .reset_index()
)


Why this matters

- Payment stress is a direct churn predictor

- Severe delays indicate credit risk

#### Credit Profile Features

In [71]:
credit_features = credit[[
    "customer_id",
    "credit_limit",
    "credit_score",
    "utilization_ratio",
    "high_utilization_flag"
]]


#### Customer Tenure & Demographics

In [72]:
customer_features = customers[[
    "customer_id",
    "age",
    "gender",
    "city",
    "account_tenure_months"
]]


#### Merge All Features → Final Analytical Dataset


In [73]:
customer_analytics = (
    customer_features
    .merge(credit_features, on="customer_id", how="left")
    .merge(txn_features, on="customer_id", how="left")
    .merge(monthly_trend, on="customer_id", how="left")
    .merge(category_features, on="customer_id", how="left")
    .merge(repayment_features, on="customer_id", how="left")
)


In [74]:
customer_analytics.fillna(0, inplace=True)


#### Derived Profitability Indicators

In [75]:
customer_analytics["net_spend_to_limit_ratio"] = (
    customer_analytics["total_spend"] /
    customer_analytics["credit_limit"].replace(0, 1)
)


In [76]:
customer_analytics["repaid_ratio"] = (
    customer_analytics["total_repaid"] /
    customer_analytics["total_spend"].replace(0, 1)
)


#### Churn Risk Flags (Rule-Based)

In [77]:
customer_analytics["churn_risk_flag"] = (
    (customer_analytics["spend_change_ratio"] < -0.3) |
    (customer_analytics["avg_payment_delay"] > 10) |
    (customer_analytics["high_utilization_flag"] == True)
)


In [78]:
customer_analytics["high_value_customer_flag"] = (
    customer_analytics["total_spend"] >
    customer_analytics["total_spend"].quantile(0.90)
)


#### Final Sanity Check

In [79]:
customer_analytics.shape
customer_analytics.head()
customer_analytics.describe()


,customer_id,age,account_tenure_months,credit_limit,credit_score,utilization_ratio,total_spend,avg_transaction_value,max_transaction_value,transaction_count,...,spend_online shopping,spend_travel,spend_utilities,total_repaid,avg_payment_delay,max_payment_delay,delayed_payment_count,severe_delay_count,net_spend_to_limit_ratio,repaid_ratio
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,100999.500000,45.235000,62.804500,144712.500000,606.706000,0.624025,22064.03502,3691.837489,9269.814630,6.000000,...,2268.914085,2155.583145,2204.201675,25479.420765,7.939899,14.360000,1.439500,0.367000,0.220055,14.254102
std,577.494589,14.044942,32.919905,84817.904255,169.239888,0.329189,15714.59622,2374.387478,10586.680365,2.451531,...,4994.294483,6353.896629,4016.853527,18029.959006,7.261384,11.640797,1.201691,0.605388,0.235758,378.052860
min,100000.000000,21.000000,6.000000,50000.000000,300.000000,0.050000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,100499.750000,33.000000,34.000000,75000.000000,467.000000,0.350000,12078.46750,2472.825679,5305.415000,4.000000,...,0.000000,0.000000,0.000000,12353.490000,1.666667,5.000000,1.000000,0.000000,0.077250,0.522378
50%,100999.500000,46.000000,63.000000,100000.000000,606.000000,0.620000,19789.48500,3361.424583,7821.645000,6.000000,...,0.000000,0.000000,0.000000,22789.730000,7.500000,15.000000,1.000000,0.000000,0.152286,1.131035
75%,101499.250000,57.000000,92.000000,200000.000000,754.000000,0.900000,28457.34250,4470.098214,10915.585000,8.000000,...,2853.447500,2395.045000,2851.597500,36484.905000,12.500000,30.000000,2.000000,1.000000,0.287593,2.201329
max,101999.000000,69.000000,119.000000,300000.000000,899.000000,1.200000,249957.82000,49991.564000,237645.000000,16.000000,...,124362.440000,152767.070000,30710.720000,93466.210000,30.000000,30.000000,7.000000,4.000000,4.999156,13946.800000


## Churn Risk Signal Preparation (Pre-Model)

#### Define Business-Driven Churn Signals

I create a interpretable, explainable churn indicators, not a black box.

Core churn dimensions:

- Engagement drop

- Payment stress

- Credit stress

- Low usage / inactivity

#### Engagement Decline Signals
Spend drop flag
- Customers whose spending has significantly declined.

In [80]:
customer_analytics["spend_drop_flag"] = (
    customer_analytics["spend_change_ratio"] < -0.30
)


Low recent activity flag

In [81]:
customer_analytics["low_recent_spend_flag"] = (
    customer_analytics["last_month_spend"] <
    customer_analytics["avg_monthly_spend"] * 0.5
)


#### Payment Stress Signals
Frequent delay flag

In [82]:
customer_analytics["frequent_delay_flag"] = (
    customer_analytics["delayed_payment_count"] >= 2
)


Severe delay flag (already engineered, reused)

In [83]:
customer_analytics["severe_delay_flag"] = (
    customer_analytics["max_payment_delay"] >= 30
)


- Payment friction strongly correlates with churn & risk

- Simple thresholds → explainable to business

#### Credit Stress Signals
High utilization flag (already available)

In [84]:
customer_analytics["credit_stress_flag"] = (
    customer_analytics["utilization_ratio"] > 0.80
)


Low credit score flag

In [85]:
customer_analytics["low_credit_score_flag"] = (
    customer_analytics["credit_score"] < 600
)


- High utilization + low score = frustration + risk

- Often leads to card abandonment

#### Inactivity / Low Engagement Signals
Very low transaction count

In [86]:
customer_analytics["low_transaction_flag"] = (
    customer_analytics["transaction_count"] <= 2
)


Zero recent activity

In [87]:
customer_analytics["no_recent_activity_flag"] = (
    customer_analytics["last_month_spend"] == 0
)


#### Combine Signals into a Churn Risk Score (Rule-Based)
Binary scoring (simple & explainable)

In [88]:
churn_signal_cols = [
    "spend_drop_flag",
    "low_recent_spend_flag",
    "frequent_delay_flag",
    "severe_delay_flag",
    "credit_stress_flag",
    "low_credit_score_flag",
    "low_transaction_flag",
    "no_recent_activity_flag"
]

customer_analytics["churn_signal_count"] = (
    customer_analytics[churn_signal_cols].sum(axis=1)
)


#### Define Churn Risk Levels

In [89]:
customer_analytics["churn_risk_level"] = pd.cut(
    customer_analytics["churn_signal_count"],
    bins=[-1, 1, 3, 8],
    labels=["Low Risk", "Medium Risk", "High Risk"]
)


#### High-Value + High-Risk Identification (Business Critical)

In [90]:
customer_analytics["priority_retention_flag"] = (
    (customer_analytics["high_value_customer_flag"] == True) &
    (customer_analytics["churn_risk_level"] == "High Risk")
)


- These customers drive revenue

- Losing them hurts most

- Primary target for retention offers

#### Final Validation & Distribution Check

In [91]:
customer_analytics["churn_risk_level"].value_counts()


churn_risk_level
Medium Risk    964
Low Risk       622
High Risk      414
Name: count, dtype: int64

In [92]:
customer_analytics[
    ["churn_signal_count", "churn_risk_level", "priority_retention_flag"]
].head()


,churn_signal_count,churn_risk_level,priority_retention_flag
0,2,Medium Risk,False
1,5,High Risk,False
2,2,Medium Risk,False
3,4,High Risk,False
4,1,Low Risk,False


## Data Validation (Final Check)

In [93]:
print("Dataset Shape:", customer_analytics.shape)

print(
    "Unique customers:",
    customer_analytics["customer_id"].nunique()
)


Dataset Shape: (2000, 46)
Unique customers: 2000


In [94]:
duplicate_customers = customer_analytics["customer_id"].duplicated().sum()
print("Duplicate customer_ids:", duplicate_customers)


Duplicate customer_ids: 0


In [95]:
missing_summary = customer_analytics.isna().sum()

missing_summary[missing_summary > 0]


Series([], dtype: int64)

In [96]:
customer_analytics.dtypes


customer_id                          int64
age                                  int64
gender                      string[python]
city                        string[python]
account_tenure_months                int64
credit_limit                       float64
credit_score                       float64
utilization_ratio                  float64
high_utilization_flag                 bool
total_spend                        float64
avg_transaction_value              float64
max_transaction_value              float64
transaction_count                  float64
high_value_txn_count               float64
avg_monthly_spend                  float64
last_month_spend                   float64
first_month_spend                  float64
spend_change_ratio                 float64
spend_dining                       float64
spend_electronics                  float64
spend_entertainment                float64
spend_fuel                         float64
spend_groceries                    float64
spend_onlin

In [97]:
assert (customer_analytics["total_spend"] >= 0).all(), "Negative total_spend found"
assert (customer_analytics["total_repaid"] >= 0).all(), "Negative total_repaid found"
assert (customer_analytics["credit_limit"] >= 0).all(), "Negative credit_limit found"


In [98]:
assert (customer_analytics["utilization_ratio"] >= 0).all(), "Negative utilization found"


In [99]:
assert (customer_analytics["avg_payment_delay"] >= 0).all(), "Negative delay found"
assert (customer_analytics["max_payment_delay"] >= 0).all(), "Negative delay found"


In [100]:
signal_cols = [
    "spend_drop_flag",
    "low_recent_spend_flag",
    "frequent_delay_flag",
    "severe_delay_flag",
    "credit_stress_flag",
    "low_credit_score_flag",
    "low_transaction_flag",
    "no_recent_activity_flag"
]

calculated_signal_sum = customer_analytics[signal_cols].sum(axis=1)

assert (
    calculated_signal_sum == customer_analytics["churn_signal_count"]
).all(), "Churn signal count mismatch"


In [101]:
customer_analytics["churn_risk_level"].value_counts(normalize=True)


churn_risk_level
Medium Risk    0.482
Low Risk       0.311
High Risk      0.207
Name: proportion, dtype: float64

In [102]:
pd.crosstab(
    customer_analytics["high_value_customer_flag"],
    customer_analytics["churn_risk_level"]
)


churn_risk_level,Low Risk,Medium Risk,High Risk
high_value_customer_flag,,,
False,558,867,375
True,64,97,39


In [103]:
customer_analytics.sample(5, random_state=42)


,customer_id,age,gender,city,account_tenure_months,credit_limit,credit_score,utilization_ratio,high_utilization_flag,total_spend,...,low_recent_spend_flag,frequent_delay_flag,severe_delay_flag,credit_stress_flag,low_credit_score_flag,low_transaction_flag,no_recent_activity_flag,churn_signal_count,churn_risk_level,priority_retention_flag
1860,101860,58,Male,Kolkata,119,200000.0,799.0,0.70,False,33328.14,...,False,False,False,False,False,False,False,0,Low Risk,False
353,100353,64,Male,Pune,26,100000.0,616.0,0.66,False,24056.89,...,False,False,False,False,False,False,False,0,Low Risk,False
1333,101333,66,Male,Bengaluru,48,200000.0,573.0,0.39,False,19944.52,...,False,False,False,False,True,False,False,2,Medium Risk,False
905,100905,67,Male,Pune,9,200000.0,606.0,0.12,False,5041.83,...,False,False,False,False,False,False,False,0,Low Risk,False
1289,101289,34,Male,Mumbai,75,200000.0,795.0,0.32,False,13368.35,...,True,False,False,False,False,False,False,2,Medium Risk,False


In [104]:
customer_analytics["data_validation_passed"] = True


## Output of This Phase – Persist Clean & Processed Data

In [105]:
import os

base_path = "output"

folders = [
    "output/cleaned",
    "output/processed",
    "output/final"
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)


#### Save Cleaned Raw Tables

In [106]:
customers.to_csv(
    "output/cleaned/customers_clean.csv",
    index=False
)

credit.to_csv(
    "output/cleaned/credit_profiles_clean.csv",
    index=False
)

transactions.to_csv(
    "output/cleaned/transactions_clean.csv",
    index=False
)

repayments.to_csv(
    "output/cleaned/repayments_clean.csv",
    index=False
)


#### Save Processed / Feature Tables

In [107]:
txn_features.to_csv(
    "output/processed/transaction_features.csv",
    index=False
)

repayment_features.to_csv(
    "output/processed/repayment_features.csv",
    index=False
)

monthly_trend.to_csv(
    "output/processed/monthly_spend_trends.csv",
    index=False
)

category_features.to_csv(
    "output/processed/category_spend_features.csv",
    index=False
)


#### Save Final Analysis-Ready Dataset

In [108]:
customer_analytics.to_csv(
    "output/final/customer_churn_profitability_dataset.csv",
    index=False
)


In [109]:
print("Final dataset shape:", customer_analytics.shape)
print("Files written successfully.")


Final dataset shape: (2000, 47)
Files written successfully.
